In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tkinter import Tk, filedialog, Label, Button, Text, Scrollbar, Frame, Toplevel, Entry, StringVar, messagebox
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
# from category_encoders import OrdinalEncoder, OneHotEncoder
from warnings import filterwarnings

filterwarnings("ignore")

# متغيرات البيانات
df, X, y, X_train, X_test, y_train, y_test = None, None, None, None, None, None, None
current_figure = None  # لتخزين الرسم الحالي

# دالة لتحميل الملف
def load_file():
    global df
    file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
    if file_path:
        try:
            df = pd.read_csv(file_path)
            result_text.insert("end", "File loaded successfully!\n")
            clean_button.pack(pady=10)  # عرض زر تنظيف البيانات
        except Exception as e:
            result_text.insert("end", f"Error loading file: {e}\n")

# دالة لتنظيف البيانات
def preprocess_data():
    global X, y, X_train, X_test, y_train, y_test
    if df is None:
        messagebox.showwarning("No Data", "Please load a CSV file first!")
        return
    try:
        cat_cols = ['sex', 'children', 'smoker', 'region']
        df[cat_cols] = df[cat_cols].astype("category")

        Q1 = df.bmi.quantile(.25)
        Q3 = df.bmi.quantile(.75)
        IQR = Q3 - Q1
        upper = Q3 + 1.5 * IQR
        outliers = df[df.bmi > upper].bmi.values
        df.bmi.replace(outliers, upper, inplace=True)

        df.drop_duplicates(keep="first", inplace=True)

        X = df.iloc[:, :-1]
        y = df.iloc[:, -1:]

        cols_to_norm = X.select_dtypes("number").columns
        scaler = MinMaxScaler()
        X[cols_to_norm] = scaler.fit_transform(X[cols_to_norm])

        cols_to_encode = ['sex', 'smoker']
        ordinal_encoder = OrdinalEncoder(cols=cols_to_encode)
        X = ordinal_encoder.fit_transform(X)

        encoder = OneHotEncoder(cols=["region"])
        X = encoder.fit_transform(X)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        result_text.insert("end", "Data cleaned and preprocessed successfully!\n")
        save_cleaned_button.pack(pady=10)  # عرض زر حفظ البيانات النظيفة
        linear_button.pack(pady=10)       # عرض زر الانحدار الخطي
        poly_button.pack(pady=10)         # عرض زر الانحدار متعدد الحدود
    except Exception as e:
        result_text.insert("end", f"Error cleaning data: {e}\n")

# دالة لحفظ البيانات النظيفة
def save_cleaned_data():
    try:
        if df is None:
            messagebox.showwarning("No Data", "No data to save. Please clean the data first.")
            return

        file_path = filedialog.asksaveasfilename(defaultextension=".xlsx",
                                                 filetypes=[("Excel files", "*.xlsx")])
        if file_path:
            df.to_excel(file_path, index=False)
            result_text.insert("end", f"Cleaned data saved to {file_path}\n")
    except Exception as e:
        result_text.insert("end", f"Error saving cleaned data: {e}\n")

# دالة لحفظ الرسم البياني كصورة
def save_plot():
    if current_figure is None:
        messagebox.showwarning("No Plot", "No plot to save. Please run a regression analysis first.")
        return

    file_path = filedialog.asksaveasfilename(defaultextension=".png",
                                             filetypes=[("PNG files", "*.png")])
    if file_path:
        current_figure.savefig(file_path)
        result_text.insert("end", f"Plot saved to {file_path}\n")

# دوال الانحدار
def run_linear_regression():
    global current_figure
    try:
        model = LinearRegression()
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        result_text.insert("end", f"Linear Regression Results:\nMSE: {mse:.2f}\nR2: {r2:.2f}\n")
        current_figure = plot_results(y_test, y_pred, title="Linear Regression Results")
        save_plot_button.pack(pady=10)  # عرض زر حفظ الرسم البياني
    except Exception as e:
        result_text.insert("end", f"Error in Linear Regression: {e}\n")

def run_polynomial_regression():
    global current_figure
    try:
        degree = int(degree_var.get())
        if degree < 1 or degree > 20:
            messagebox.showwarning("Invalid Degree", "Please enter a degree between 1 and 20.")
            return

        poly = PolynomialFeatures(degree=degree, include_bias=False)
        X_train_poly = poly.fit_transform(X_train)
        X_test_poly = poly.transform(X_test)

        model = LinearRegression()
        model.fit(X_train_poly, y_train)

        y_pred = model.predict(X_test_poly)

        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        result_text.insert("end", f"Polynomial Regression Results (Degree={degree}):\nMSE: {mse:.2f}\nR2: {r2:.2f}\n")
        current_figure = plot_results(y_test, y_pred, title=f"Polynomial Regression (Degree={degree})")
        save_plot_button.pack(pady=10)  # عرض زر حفظ الرسم البياني
    except Exception as e:
        result_text.insert("end", f"Error in Polynomial Regression: {e}\n")

def plot_results(y_test, y_pred, title):
    fig, ax = plt.subplots(figsize=(5, 4))
    ax.scatter(y_test, y_pred, alpha=0.5)
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--', linewidth=2)
    ax.set_title(title)
    ax.set_xlabel("Actual Charges")
    ax.set_ylabel("Predicted Charges")

    canvas = FigureCanvasTkAgg(fig, master=plot_frame)
    canvas.draw()
    canvas.get_tk_widget().pack()
    return fig

# واجهة المستخدم
root = Tk()
root.title("Regression GUI")
root.geometry("900x700")

Label(root, text="Regression GUI", font=("Arial", 16)).pack(pady=10)

load_button = Button(root, text="Load CSV File", command=load_file, width=20)
load_button.pack(pady=10)

clean_button = Button(root, text="Clean Data", command=preprocess_data, width=20)
save_cleaned_button = Button(root, text="Save Cleaned Data", command=save_cleaned_data, width=20)
linear_button = Button(root, text="Run Linear Regression", command=run_linear_regression, width=20)
poly_button = Button(root, text="Run Polynomial Regression", command=run_polynomial_regression, width=25)
save_plot_button = Button(root, text="Save Plot", command=save_plot, width=20)

Label(root, text="Enter Degree for Polynomial Regression:", font=("Arial", 12)).pack(pady=5)
degree_var = StringVar(value="2")
degree_entry = Entry(root, textvariable=degree_var, width=10)
degree_entry.pack()

result_frame = Frame(root)
result_frame.pack(side="left", fill="y", padx=10)

result_text = Text(result_frame, wrap="word", width=50, height=30)
result_text.pack(side="left", fill="y")

scrollbar = Scrollbar(result_frame, command=result_text.yview)
scrollbar.pack(side="right", fill="y")
result_text.config(yscrollcommand=scrollbar.set)

plot_frame = Frame(root)
plot_frame.pack(side="right", fill="both", expand=True, padx=10, pady=10)

root.mainloop()


F1 Score: 0.8
